# MSDS697 Group Project

### -- Using SparkML to Predict New York City Taxi Fare --

### Group: Jetlag
### Group Member: Jenny Kong, Joyce Chang, Xinke Sun, Zhe Yuan

### 1. Load data from MongoDB

In [1]:
import os

pyspark_submit_args = '--packages org.mongodb.spark:mongo-spark-connector_2.11:2.4.0 pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("myApp").config("spark.mongodb.input.uri",\
                                             "mongodb://54.201.216.215/mydb.option").getOrCreate()    
# 54.201.216.215 -- mongos master public ID

In [3]:
raw = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()
raw = raw.drop("_id")

In [4]:
from pyspark import SparkContext
from pyspark.sql.types import *

sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()

### 2. Data Cleaning, Change DataType and Create Dataframe

In [5]:
raw_rdd = raw.rdd

In [6]:
# filter fare which are less than 5.0 (initial charge)
fare_rdd = raw_rdd.filter(lambda x: x[2] >= 5.0)

In [7]:
# filter location which are not belong to NYC
location_rdd = fare_rdd.filter(lambda x: x[1]> -75 and x[1]< -73
                              ).filter(lambda x: x[0]> 39 and x[0]< 43
                                      ).filter(lambda x: x[7]> -75 and x[7]< -73
                                              ).filter(lambda x: x[6]> 39 and x[6]< 43)

In [8]:
# add 'time of day', drop original date
# features : time, pickup_datetime, up_lat, up_lon, drop_lat, drop_lon, passenger_count, fare_amount
final_rdd = location_rdd.map(lambda x: (x[3].split()[1].split(".")[0].split(":")[0],
                                        x[5].split()[0], x[6], x[7], x[0], x[1], x[4],x[2]))

In [10]:
# create dataframe
def IntegerSafe(value): 
    try:
        return int(value)
    except ValueError:
        return None 

taxiSchema = StructType([StructField('hour_of_day', IntegerType(), False), 
                         StructField('pickup_datetime', StringType(), False),
                         StructField('pickup_latitude', DoubleType(), False),
                         StructField('pickup_longitude', DoubleType(), False), 
                         StructField('dropoff_latitude', DoubleType(), False), 
                         StructField('dropoff_longitude', DoubleType(), False),
                         StructField('passenger_count', IntegerType(), False),
                         StructField('fare_amount', DoubleType(), False)
                         ])

taxi_df = ss.createDataFrame(final_rdd.map(lambda x : (IntegerSafe(x[0]),x[1],x[2],
                                                       x[3],x[4],x[5],x[6],x[7])),taxiSchema)

In [12]:
taxi_df.show(5)

+-----------+---------------+---------------+----------------+----------------+-----------------+---------------+-----------+
|hour_of_day|pickup_datetime|pickup_latitude|pickup_longitude|dropoff_latitude|dropoff_longitude|passenger_count|fare_amount|
+-----------+---------------+---------------+----------------+----------------+-----------------+---------------+-----------+
|          4|     2012-04-21|      40.733143|       -73.98713|       40.758092|       -73.991567|              1|        7.7|
|         20|     2012-11-20|      40.751662|      -73.980002|       40.764842|       -73.973802|              1|        7.5|
|         13|     2012-12-03|      40.726713|      -74.006462|       40.731628|       -73.993078|              1|        9.0|
|         19|     2013-07-02|      40.728867|       -74.00536|       40.710907|       -74.008913|              1|        7.0|
|         17|     2011-04-05|      40.737547|      -74.001821|       40.722788|        -73.99806|              2|     

In [13]:
from datetime import datetime
from pyspark.sql.functions import *

str_date_func =  udf(lambda x: datetime.strptime(x, '%Y-%m-%d'), DateType())
taxi_df = taxi_df.withColumn('date', str_date_func(taxi_df['pickup_datetime'])).drop("pickup_datetime")
taxi_df = taxi_df.withColumnRenamed('date', 'pickup_datetime')

In [14]:
taxi_df.printSchema()

root
 |-- hour_of_day: integer (nullable = false)
 |-- pickup_latitude: double (nullable = false)
 |-- pickup_longitude: double (nullable = false)
 |-- dropoff_latitude: double (nullable = false)
 |-- dropoff_longitude: double (nullable = false)
 |-- passenger_count: integer (nullable = false)
 |-- fare_amount: double (nullable = false)
 |-- pickup_datetime: date (nullable = true)



### 3. Feature Engineering

In [15]:
# create distance based on latitute and longitude
from pyspark.sql.types import *
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    try:
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1 
        dlat = lat2 - lat1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        r = 6371
        return c * r * 1000
    except AttributeError:
        return None
    
haversine_udf = udf(lambda a, b, c, d: haversine(a,b,c,d),FloatType())
dist_df = taxi_df.withColumn('dist', haversine_udf(taxi_df['pickup_longitude'], 
                              taxi_df['pickup_latitude'],
                              taxi_df['dropoff_longitude'], 
                              taxi_df['dropoff_latitude']))

In [19]:
# create time related features, including year, month, quarter, day of month, day of week
from pyspark.sql.functions import *

year_mon_day_df = dist_df.withColumn('year', year(dist_df['pickup_datetime'])
                                    ).withColumn('month', month(dist_df['pickup_datetime'])
                                                ).withColumn('quarter', quarter(dist_df['pickup_datetime'])
                                                            ).withColumn('day_of_month', dayofmonth(dist_df['pickup_datetime']))
year_mon_day_df = year_mon_day_df.withColumn("day_of_week", date_format("pickup_datetime", "u"))

In [23]:
# create rushhour feature and change data type
rushhour_udf = udf(lambda x: 1 if (7 < x <10 or 16 < x < 19) else 0)
rush_hour_df = year_mon_day_df.withColumn('rushhour', rushhour_udf('hour_of_day'))

rush_hour_float_df = rush_hour_df.withColumn('rushhour_new', rush_hour_df['rushhour'].cast(FloatType())).drop("rushhour")
rush_hour_float_df = rush_hour_float_df.withColumnRenamed('rushhour_new', 'rushhour')

In [27]:
rush_hour_float_c = rush_hour_float_df.cache()

In [28]:
# change data type : day_of_week
day_week_float_df = rush_hour_float_c.withColumn('day_of_week_new', rush_hour_float_c['day_of_week'].cast(FloatType())).drop("day_of_week")
day_week_float_df = day_week_float_df.withColumnRenamed('day_of_week_new', 'day_of_week')

In [30]:
# manually one hot encoding for year, quarter, day_of_week
def num1(value):
    if value == 1: return 1
    else: return 0
def num2(value):
    if value == 2: return 1
    else: return 0
def num3(value):
    if value == 3: return 1
    else: return 0
def num4(value):
    if value == 4: return 1
    else: return 0
def num5(value):
    if value == 5: return 1
    else: return 0
def num6(value):
    if value == 6: return 1
    else: return 0
def num7(value):
    if value == 7: return 1
    else: return 0

num1_udf = udf(lambda x: num1(x))
num2_udf = udf(lambda x: num2(x))
num3_udf = udf(lambda x: num3(x))
num4_udf = udf(lambda x: num4(x))
num5_udf = udf(lambda x: num5(x))
num6_udf = udf(lambda x: num6(x))
num7_udf = udf(lambda x: num7(x))

day_week_float_df = day_week_float_df.withColumn('day_1', num1_udf(day_week_float_df['day_of_week']))
day_week_float_df = day_week_float_df.withColumn('day_2', num2_udf(day_week_float_df['day_of_week']))
day_week_float_df = day_week_float_df.withColumn('day_3', num3_udf(day_week_float_df['day_of_week']))
day_week_float_df = day_week_float_df.withColumn('day_4', num4_udf(day_week_float_df['day_of_week']))
day_week_float_df = day_week_float_df.withColumn('day_5', num5_udf(day_week_float_df['day_of_week']))
day_week_float_df = day_week_float_df.withColumn('day_6', num6_udf(day_week_float_df['day_of_week']))
day_week_float_df = day_week_float_df.withColumn('day_7', num7_udf(day_week_float_df['day_of_week']))

quarter_df = day_week_float_df.withColumn('quarter_1', num1_udf(day_week_float_df['quarter']))
quarter_df = quarter_df.withColumn('quarter_2', num2_udf(quarter_df['quarter']))
quarter_df = quarter_df.withColumn('quarter_3', num3_udf(quarter_df['quarter']))
quarter_df = quarter_df.withColumn('quarter_4', num4_udf(quarter_df['quarter']))


In [31]:
def year1(value):
    if value == 2009: return 1
    else: return 0

def year2(value):
    if value == 2010: return 1
    else: return 0
    
def year3(value):
    if value == 2011: return 1
    else: return 0
    
def year4(value):
    if value == 2012: return 1
    else: return 0

def year5(value):
    if value == 2013: return 1
    else: return 0
    
def year6(value):
    if value == 2014: return 1
    else: return 0
    
def year7(value):
    if value == 2015: return 1
    else: return 0


year1_udf = udf(lambda x: year1(x))
year2_udf = udf(lambda x: year2(x))
year3_udf = udf(lambda x: year3(x))
year4_udf = udf(lambda x: year4(x))
year5_udf = udf(lambda x: year5(x))
year6_udf = udf(lambda x: year6(x))
year7_udf = udf(lambda x: year7(x))

year_df = quarter_df.withColumn('year_1', year1_udf(quarter_df['year']))
year_df = year_df.withColumn('year_2', year2_udf(quarter_df['year']))
year_df = year_df.withColumn('year_3', year3_udf(quarter_df['year']))
year_df = year_df.withColumn('year_4', year4_udf(quarter_df['year']))
year_df = year_df.withColumn('year_5', year5_udf(quarter_df['year']))
year_df = year_df.withColumn('year_6', year6_udf(quarter_df['year']))
year_df = year_df.withColumn('year_7', year7_udf(quarter_df['year']))

In [ ]:
year_df_c = year_df.cache()

In [34]:
# change data type for day_1~7
day = ["day_1", "day_2", "day_3", "day_4", "day_5", "day_6", "day_7"]
for d in day:
    year_df_c = year_df_c.withColumn(d + "_new", year_df_c[d].cast(FloatType())).drop(d)
    year_df_c = year_df_c.withColumnRenamed(d + "_new", d)

In [35]:
day_float_c = year_df_c.cache()

In [36]:
# change data type for quarter
quarter = ["quarter_1", "quarter_2", "quarter_3", "quarter_4"]
for q in quarter:
    day_float_c = day_float_c.withColumn(q + "_new", day_float_c[q].cast(FloatType())).drop(q)
    day_float_c = day_float_c.withColumnRenamed(d + "_new", q)

In [ ]:
quarter_float_c = day_float_c.cache()

In [38]:
# change data type for year
year = ["year_1", "year_2", "year_3", "year_4", "year_5", "year_6", "year_7"]
for y in year:
    quarter_float_c = quarter_float_c.withColumn(y + "_new", quarter_float_c[y].cast(FloatType())).drop(y)
    quarter_float_c = quarter_float_c.withColumnRenamed(y + "_new", y)

In [ ]:
year_float_c = quarter_float_c.cache()

### 4. Vector Assembler

In [40]:
from pyspark.ml.feature import VectorAssembler

cols = list(set(year_float_c.columns) - {'fare_amount','quarter','dropoff_longitude','month',\
                                  'pickup_datetime','dropoff_latitude','pickup_latitude',\
                                  'day_of_week','year','pickup_longitude'})
va = VectorAssembler(outputCol="features", inputCols = cols)
va_df = va.transform(year_float_c).select("features", "fare_amount").withColumnRenamed("fare_amount", "label")

In [41]:
va_df.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(23,[0,1,2,5,7,13...|  7.7|
|(23,[0,1,2,7,15,1...|  7.5|
|(23,[0,1,2,7,9,16...|  9.0|
|(23,[0,1,2,4,12,1...|  7.0|
|(23,[0,1,2,5,10,1...|  7.7|
|(23,[0,1,2,4,15,1...| 10.5|
|(23,[0,1,2,3,16,2...|  6.1|
|(23,[0,1,2,5,10,1...|  7.3|
|(23,[0,1,2,5,7,17...|  9.3|
|(23,[0,1,2,5,6,18...|22.54|
|(23,[0,1,2,14,17,...| 31.9|
|(23,[0,1,2,4,8,10...| 18.1|
|(23,[0,1,2,6,12,1...|  9.0|
|(23,[0,1,2,9,14,2...|  9.8|
|(23,[0,1,2,10,13,...| 10.9|
|(23,[0,1,2,4,8,10...|  6.9|
|(23,[0,1,2,12,14,...|  9.0|
|(23,[0,1,2,10,15,...|  9.7|
|(23,[0,1,2,4,12,1...|  7.5|
|(23,[0,1,2,7,8,14...| 15.3|
+--------------------+-----+
only showing top 20 rows



### 5. Train Test Split

In [42]:
splits = va_df.randomSplit([0.8, 0.2])
train_df = splits[0].cache()
test_df = splits[1].cache()

### 6. Fit Multiple Regression Models, Do Transformation and Evaluation

#### Linear Regression

In [45]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

lr = LinearRegression(featuresCol = 'features', labelCol='label', maxIter=10, regParam=0.3, elasticNetParam=0.8)

#fit model to train data
lr_model = lr.fit(train_df)

#do prediction on test dataset
lr_predictions = lr_model.transform(test_df)

#check prediction outcome
lr_predictions.select("prediction","label","features").show(5)

#Evaluate the model
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="label",metricName="r2")

print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

test_result = lr_model.evaluate(test_df) 
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

R Squared (R2) on test data = 0.585697
Root Mean Squared Error (RMSE) on test data = 6.42915


#### Decision Tree

In [47]:
from pyspark.ml.regression import DecisionTreeRegressor

dt = DecisionTreeRegressor(featuresCol="features")
dt_model = dt.fit(train_df)
dt_prediction = dt_model.transform(test_df)
dt_prediction_c = dt_prediction.cache()

dt_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="r2")
print("R Squared (R2) on test data = %g" % dt_evaluator.evaluate(dt_prediction))

dt_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="rmse")
print(“Root Mean Squared Error (RMSE) on test data = %g” % dt_evaluator.evaluate(dt_prediction_c))

R Squared (R2) on test data = 0.721088
Root Mean Squared Error (RMSE) on test data = 5.89855


#### Random Forest

In [48]:
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(featuresCol="features")
rf_model = rf.fit(train_df)
rf_prediction = rf_model.transform(test_df)
rf_prediction_c = rf_prediction.cache()

rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName='rmse')
print("Root Mean Squared Error (RMSE) on test data = %g" % rf_evaluator.evaluate(rf_prediction_c))

Root Mean Squared Error (RMSE) on test data = 5.27507
